# Working with existing Data

This notebook explores the use of folium with existing data, reading a json file.

In [10]:
from itertools import chain as ch
import folium
import json
import pathlib

A function to transfer data from a json file, using python context.

In [11]:
def JSON_Transfer(path_):  
    '''
    Return a dict or a list of dicts from a given JSON.
    '''
    with open(path_) as j:

        return json.load(j)

There is a piece of json data on the assets folder locatade on the main repository.

In [14]:
# Insert the path to your file here.
path = pathlib.Path('Assets/pocos_Parnaiba.json')
data = JSON_Transfer(path)

#### Printing the data

In [ ]:
print(data)

#### Maping the data.

Exploring adiotonal features like styles and features groups.ˆ

In [17]:
# - Basic instance of the folium object.
start_coords = (-5.42381, -46.09992)
folium_map = folium.Map(location=start_coords, zoom_start=7, name='MabaBase',)
# - Creating a feature group for all the well objects.
pocos = folium.FeatureGroup(name='Poços Processados Bacia do Parnaíba')
# - Creating a feature group for the thickness of the codo layer.
codolayer = folium.FeatureGroup(name='Expessura CODO')
# - Creating styles
style1 = {'fillColor': '#0e8a23', 'lineColor': '#0e8a23'}
style2 = {'fillColor': '#17517', 'lineColor': '#0e8a23'}

for well in data:
    # - getting the well name
    tooltip = well['POCO']
    form = well[tooltip]
    # - Strong typing the variable
    form: dict
    # - A list for the geological unity names
    stringsM = []
    lat = well['LATITUDE'][0:9]
    long = well['LONGITUDE'][0:9]
    ######################################################
    # - Acessing the dictionary holding the formations - #
    # - Also creating a custom html div                - #
    ######################################################

    for g, v in form.items():
        try:
            # Floating values for calculations
            topo = float(v['TOPO COTA'].rstrip().lstrip())
            base = float(v['BASE COTA'].rstrip().lstrip())
            # - The radius gets the unity depth.
            radius = (base-topo)*(-1)

        # - Error Handling.
        except ValueError as v:
            #prints a value error.
            print(f"The current well:{well['POCO']}, has presented {v}")
            pass
        except Exception as e:
            #prints a generic error error.
            print(f"The current well:{well['POCO']}, has presented {e}")
            pass

        # - Creating custom HTML div for the popup
        # - First a p tag for each unity
        strings = f"<p style = 'font-size=0.01rem; margin:0;'> {g} cota = {radius} </p>"
        # - 
        stringsM.append(strings)
        div = f"<div style='overflow:scroll; max-height:300px;width:300px;'>{stringsM}</div>"
        
        # - Filtering for a specifiv unity.
        if str(g).lstrip().rstrip().__contains__('CODO'):
            # - Safe execution part, with a try block:
            try:
                # - FLoating the numbers
                topo = float(v['TOPO COTA'].rstrip().lstrip())
                base = float(v['BASE COTA'].rstrip().lstrip())
                # - Creating circles for the depth of a geological unity
                folium.Circle(
                    [well['LATITUDE'][0:9], well['LONGITUDE'][0:9]],
                    radius=(base-topo)*(-60),

                    popup=f"{g}",

                ).add_to(codolayer)

            # - Error Handling.
            except ValueError as v:
                #prints a value error.
                print(f"The current well:{well['POCO']}, has presented {v}")
                pass
            except Exception as e:
                #prints a generic error error.
                print(f"The current well:{well['POCO']}, has presented {e}")
                pass
    ###############################################
    # - Creating a marker for each well         - #
    # - And appending the div as a custom popup - #
    ###############################################
    markerpoco = folium.Marker(
        [lat, long],
        popup=div,
        tooltip=tooltip, icon=folium.Icon(icon="angle-down",
                                          name='Poços',)
    )
    pocos.add_child(markerpoco)
#################################
# - Adding the features group - #
#################################
folium_map.add_child(pocos)
folium_map.add_child(codolayer)
# - adding layer control
folium.LayerControl().add_to(folium_map)
# - If you want to save it, uncomment
#folium_map.save()
folium_map
